In [1]:
from imblearn.metrics import geometric_mean_score
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn import metrics
np.random.seed(13)
import time
import sys
from datetime import datetime, timedelta
from sklearn import svm
import pandas as pd
import binaryFn
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
files=['AAPL','AMZN','PEP','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX','BKNG','ADBE','AMGN','TXN','AVGO','PYPL','GILD','COST','QCOM']       
   
prices=list()
sentiment=list()
for file in files:
    param = {
        'q': file, # Stock symbol (ex: "AAPL")
        'i': "900", # Interval size in seconds ("86400" = 1 day intervals)
        'x': "NASD", # INDEXNASDAQStock exchange symbol on which stock is traded (ex: "NASD")
        'p': "2Y" # Period (Ex: "1Y" = 1 year)
        }
    sentimentVector =  pd.read_csv('/home/andrea/Desktop/NLFF/intrinioDatasetUpdated/Sentiment/'+file+'.csv')
    price= pd.read_csv('/home/andrea/Desktop/NLFF/TechnicalDatasetUpdated/'+file+'.csv')
    #alignment of data
    maxdata=max(sentimentVector['initTime'])
    mindata=min(sentimentVector['initTime'])
    price['Unnamed: 0'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in price['Unnamed: 0']]
    price['Unnamed: 0'] = [row-timedelta(hours=8) for row in price['Unnamed: 0']]
    price=price[price['Unnamed: 0']>=mindata]
    price=price[price['Unnamed: 0']<=maxdata]
    print(sentimentVector[0:10])
    print(price[0:10])
    sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)
    meanvector=np.concatenate((sentimentVector,pd.rolling_mean(sentimentVector,5)),axis=1)
    meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,10)),axis=1)
    meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,15)),axis=1)
    meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,20)),axis=1)
    price=price.drop(['Unnamed: 0'],axis=1)
    #['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
    #df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
    #df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
    #
    sentiment.append(meanvector)
    prices.append(price)

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00         0.000      0.000     0.000   
1           2  2017-08-14 14:00:00         0.000      0.000     0.000   
2           3  2017-08-14 14:15:00         0.000      0.000     1.000   
3           4  2017-08-14 14:30:00         0.000      0.000     1.000   
4           5  2017-08-14 14:45:00         0.250      0.250     1.250   
5           6  2017-08-14 15:00:00         1.125      0.625     1.125   
6           7  2017-08-14 15:15:00         1.125      1.625     3.125   
7           8  2017-08-14 15:30:00         1.125      1.625     3.125   
8           9  2017-08-14 15:45:00         1.125      1.625     4.125   
9          10  2017-08-14 16:00:00         1.125      1.625     4.125   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0        0.000          0.0         0.00         0  
1       0.0        0.000          0.0         0.00         0  
2      

/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=5,center=False).mean()
/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=10,center=False).mean()
/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=15,center=False).mean()
/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=20,center=False).mean()


   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00      0.000000   0.000000  0.000000   
1           2  2017-08-14 14:00:00      0.000000   0.000000  0.333333   
2           3  2017-08-14 14:15:00      0.000000   0.250000  0.083333   
3           4  2017-08-14 14:30:00      0.000000   0.250000  2.083333   
4           5  2017-08-14 14:45:00      0.333333   0.208333  1.680556   
5           6  2017-08-14 15:00:00      0.333333   0.208333  1.680556   
6           7  2017-08-14 15:15:00      0.333333   0.208333  1.680556   
7           8  2017-08-14 15:30:00      0.333333   0.302083  1.920139   
8           9  2017-08-14 15:45:00      0.166667   0.651042  3.460069   
9          10  2017-08-14 16:00:00      0.166667   0.651042  3.460069   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0  0.000000     0.000000          0.0     0.000000         0  
1  0.333333     0.333333          0.0     0.000000         3  
2  0.58

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0        0.0       0.0   
1           2  2017-08-14 14:00:00           0.0        0.0       0.0   
2           3  2017-08-14 14:15:00           0.0        0.0       0.0   
3           4  2017-08-14 14:30:00           0.0        0.0       0.0   
4           5  2017-08-14 14:45:00           0.0        0.0       0.0   
5           6  2017-08-14 15:00:00           0.0        0.0       0.0   
6           7  2017-08-14 15:15:00           0.0        1.0       2.0   
7           8  2017-08-14 15:30:00           0.0        1.0       2.0   
8           9  2017-08-14 15:45:00           0.0        1.0       5.0   
9          10  2017-08-14 16:00:00           0.0        1.0       5.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0          0.0          0.0          0.0         0  
1       0.0          0.0          0.0          0.0         0  
2      

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00      0.000000   0.000000  0.000000   
1           2  2017-08-14 14:00:00      0.000000   0.000000  0.000000   
2           3  2017-08-14 14:15:00      0.000000   0.000000  0.000000   
3           4  2017-08-14 14:30:00      0.000000   0.000000  0.000000   
4           5  2017-08-14 14:45:00      0.333333   0.333333  1.666667   
5           6  2017-08-14 15:00:00      0.333333   0.333333  1.666667   
6           7  2017-08-14 15:15:00      0.333333   0.333333  1.666667   
7           8  2017-08-14 15:30:00      0.333333   0.333333  1.666667   
8           9  2017-08-14 15:45:00      0.333333   0.333333  3.666667   
9          10  2017-08-14 16:00:00      0.333333   0.333333  3.666667   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0     0.000000          0.0          0.0         0  
1       0.0     0.000000          0.0          0.0         0  
2      

...

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0   0.000000       0.0   
1           2  2017-08-14 14:00:00           0.0   0.000000       0.0   
2           3  2017-08-14 14:15:00           0.0   0.000000       0.0   
3           4  2017-08-14 14:30:00           0.0   0.000000       0.0   
4           5  2017-08-14 14:45:00           0.0   0.000000       1.0   
5           6  2017-08-14 15:00:00           0.0   0.000000       1.0   
6           7  2017-08-14 15:15:00           0.0   0.000000       1.0   
7           8  2017-08-14 15:30:00           0.0   0.000000       1.0   
8           9  2017-08-14 15:45:00           0.0   0.333333       2.0   
9          10  2017-08-14 16:00:00           0.0   0.333333       2.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0  0.000000          0.0          0.0     0.000000         0  
1  0.000000          0.0          0.0     0.000000         0  
2  0.00

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0        0.0       0.0   
1           2  2017-08-14 14:00:00           0.0        0.0       0.0   
2           3  2017-08-14 14:15:00           0.0        0.0       0.0   
3           4  2017-08-14 14:30:00           0.0        0.0       0.0   
4           5  2017-08-14 14:45:00           0.0        0.0       0.0   
5           6  2017-08-14 15:00:00           0.0        0.0       0.0   
6           7  2017-08-14 15:15:00           0.0        0.0       0.0   
7           8  2017-08-14 15:30:00           0.0        0.0       0.0   
8           9  2017-08-14 15:45:00           0.0        0.0       0.0   
9          10  2017-08-14 16:00:00           0.0        0.0       0.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0          0.0          0.0          0.0         0  
1       0.0          0.0          0.0          0.0         0  
2      

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0        0.0       0.0   
1           2  2017-08-14 14:00:00           0.0        0.0       0.0   
2           3  2017-08-14 14:15:00           0.0        0.0       0.0   
3           4  2017-08-14 14:30:00           0.0        0.0       0.0   
4           5  2017-08-14 14:45:00           0.0        0.0       0.0   
5           6  2017-08-14 15:00:00           0.0        0.0       0.0   
6           7  2017-08-14 15:15:00           0.0        0.0       0.0   
7           8  2017-08-14 15:30:00           0.0        0.0       0.0   
8           9  2017-08-14 15:45:00           0.0        0.0       0.0   
9          10  2017-08-14 16:00:00           0.0        0.0       0.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0          0.0          0.0          0.0         0  
1       0.0          0.0          0.0          0.0         0  
2      

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0        0.0       0.0   
1           2  2017-08-14 14:00:00           0.0        0.0       0.0   
2           3  2017-08-14 14:15:00           0.0        0.0       0.0   
3           4  2017-08-14 14:30:00           0.0        0.0       0.0   
4           5  2017-08-14 14:45:00           0.0        0.0       0.0   
5           6  2017-08-14 15:00:00           0.0        0.0       0.0   
6           7  2017-08-14 15:15:00           0.0        0.0       0.0   
7           8  2017-08-14 15:30:00           0.0        0.0       0.0   
8           9  2017-08-14 15:45:00           0.0        0.0       0.0   
9          10  2017-08-14 16:00:00           0.0        0.0       0.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0          0.0          0.0          0.0         0  
1       0.0          0.0          0.0          0.0         0  
2      

   Unnamed: 0             initTime  CONSTRAINING  LITIGIOUS  NEGATIVE  \
0           1  2017-08-14 13:45:00           0.0        0.0       0.0   
1           2  2017-08-14 14:00:00           0.0        0.0       0.0   
2           3  2017-08-14 14:15:00           0.0        0.0       0.0   
3           4  2017-08-14 14:30:00           0.0        0.0       0.0   
4           5  2017-08-14 14:45:00           0.0        0.0       0.0   
5           6  2017-08-14 15:00:00           0.0        0.0       0.0   
6           7  2017-08-14 15:15:00           0.0        1.0       2.0   
7           8  2017-08-14 15:30:00           0.0        1.0       2.0   
8           9  2017-08-14 15:45:00           0.0        2.0       3.0   
9          10  2017-08-14 16:00:00           0.0        2.0       3.0   

   POSITIVE  UNCERTAINTY  SUPERFLUOUS  INTERESTING  NUM_NEWS  
0       0.0          0.0          0.0          0.0         0  
1       0.0          0.0          0.0          0.0         0  
2      

In [2]:
def binarySearch(index,window,prices,sentiment,acc):
    pivot=math.floor(len(index)/2)
    maxind=index
    maxres=acc
    
    if(len(index)==1):
        return binaryFn.binaryFeatureSelection(index,window,sentiment,prices)  
    else:
        maxres1,maxind1=binarySearch(index[0:pivot],window,prices,sentiment,0)
        if(maxres1>acc):
            maxres=maxres1
            maxind=maxind1
        maxres2,maxind2=binarySearch(index[pivot:],window,prices,sentiment,0)
        if(maxres1>acc):
            maxres=maxres2
            maxind=maxind2
    return maxres,maxind
    

In [3]:
#price first alf
sys.setrecursionlimit(1500)
ind=range(0,math.floor(len(price)/2))

window=140
price=np.array(price)
result,bestfeature=binarySearch(ind,window,prices,sentiment,0)
print(result)
print(bestfeature)

Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADB

Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC


Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AA

Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN
Working on... AVGO
Working on... PYPL
Working on... GILD
Working on... COST
Working on... QCOM
Working on... AAPL
Working on... AMZN
Working on... PEP
Working on... GOOGL
Working on... MSFT
Working on... FB
Working on... INTC
Working on... CSCO
Working on... CMCSA
Working on... NVDA
Working on... NFLX
Working on... BKNG
Working on... ADBE
Working on... AMGN
Working on... TXN

KeyboardInterrupt: 